In [ ]:
%%capture
# Download the corresponding PyTorch Geometric module
"""
Assign to TORCH with what you get from the cell above. E.g., export TORCH=1.12.1+cu113
"""
%env TORCH=1.13.0+cu116
!pip install scipy==1.8.0
!pip install networkx
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.utils import from_networkx
from torch_geometric.loader import DataLoader
from torch_geometric.nn import Sequential, GCNConv, global_mean_pool
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from google.colab import files

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
rng = np.random.default_rng()

In [ ]:
# The range of different graph sizes
global graph_size
graph_size = 15
graph_depth = int((graph_size)**(1/2))
module = 7
# The list of all graph pairs
graphs = []

from networkx.generators.nonisomorphic_trees import nonisomorphic_trees
#tree_list = list(nonisomorphic_trees(graph_size))
tree_list = [nx.balanced_tree(r=2, h=graph_depth)]
print(tree_list[0].nodes)
di_tree_list = []
for G in tree_list:
  di_tree_list.append(nx.DiGraph([(u,v) for (u,v) in G.edges() if u<v]+[(v,u) for (u,v) in G.edges() if u>v]))


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [ ]:
def sum_tree(node, gg, t):
  s = 0
  has_neighbor = False
  for n in list(t.neighbors(node)):
    if not has_neighbor:
      has_neighbor = True
    if gg.x[n] != gg.x[node]:
      s += 1.0*sum_tree(n,gg,t)
    #elif gg.x[n] != gg.x[node]:
      #s += 2.0*sum_tree(n,gg,t)
  if has_neighbor:
    return s
  else:
    return 1.0

def sum_diff_tree(node, gg, t):
  s = 0
  has_neighbor = False
  for n in list(t.neighbors(node)):
    #print(list(t.neighbors(node)))
    if not has_neighbor:
      has_neighbor = True
    if gg.x[n] == 1.0:
    #if True:
      s += sum_tree(n,gg,t)
  if has_neighbor:
    return s
  else:
    return 1.0

def sum_1_tree(node, gg, t):
  for n in list(t.neighbors(node)):
    if gg.x[n] == gg.x[0]:
      return 1
  return 0

def sum_s_tree(node, gg, t):
  s = 0
  has_neighbor = False
  for n in list(t.neighbors(node)):
    if not has_neighbor:
      has_neighbor = True
    s += sum_tree(n,gg,t)
  if has_neighbor:
    return s
  else:
    return gg.x[node]

In [ ]:
import torch_geometric
# The list of Data objects
dataset = []
maximum = 0
# Loop over trees
for tree in di_tree_list:
  #for i in range(graph_size+1):
    #graph_geom.x = (torch.rand(size=(graph_geom.num_nodes,1)) < 0.5).float()
  for j in range(2**(graph_size)):
    graph_geom = torch_geometric.utils.from_networkx(tree)
    node_feats = list(map(float, list('{0:0b}'.format(j))))
    if len(node_feats)<graph_size:
      node_feats+=[0 for k in range(graph_size-len(node_feats))]
    #print(node_feats)
    graph_geom.x = torch.unsqueeze(torch.tensor(node_feats), 1)
    #print(graph_geom.x)
    #print(graph_geom.x)
    #graph_geom.x.requires_grad_()
    #graph_geom.y = graph_geom.x[0]
    graph_geom.y = torch.tensor([int(sum_tree(0, graph_geom, tree))])
    # Add the Data object to the dataset
    dataset.append(graph_geom.to(device))
    #graph_geom.y.requires_grad_()

    if graph_geom.y>maximum:
      maximum=graph_geom.y

      
print(len(dataset))
print(type(dataset[0].x[0][0]), dataset[0].y)
print(dataset[1000], dataset[1].y)
print(maximum)

32768
<class 'torch.Tensor'> tensor([0])
Data(edge_index=[2, 14], num_nodes=15, x=[15, 1], y=[1]) tensor([0])
tensor([8])


In [ ]:
from torch_geometric.nn import GAT

class GAT_N(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, num_layers=0, graph_size=graph_size, act_fn=nn.ReLU()):
    super().__init__()
    self.act_fn = act_fn
    self.num_layers = num_layers
    self.layers = nn.ModuleList([])
    #self.layers.append(nn.Embedding(2, hidden_dim))
    self.layer = GAT(
        input_dim, hidden_dim, num_layers, output_dim, v2=True, jk=None
    )
    #self.layers.append(nn.Linear(hidden_dim, output_dim))

  def forward(self, input_geom):
    x = input_geom.x
    e = input_geom.edge_index
    #x = self.layers[0](x.flatten())
    x = self.layer(x,e[[1,0],:])
    #x = self.layers[-1](x)
    return x
    #return nn.LogSoftmax(dim=1)(x)
    
  @staticmethod
  def _weight_reset(module):
    if isinstance(module, GAT) or isinstance(module, nn.Linear):
      module.reset_parameters()

  def reset_parameters(self):
    return self.layers.apply(type(self)._weight_reset)

class GAT_CAT(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, num_layers=0, graph_size=graph_size, act_fn=nn.ReLU()):
    super().__init__()
    self.act_fn = act_fn
    self.num_layers = num_layers
    self.layers = nn.ModuleList([])
    #self.layers.append(nn.Embedding(2, hidden_dim))
    self.layer = GAT(
        input_dim, hidden_dim, num_layers, output_dim, v2=True, jk='cat'
    )
    #self.layers.append(nn.Linear(hidden_dim, output_dim))

  def forward(self, input_geom):
    x = input_geom.x
    e = input_geom.edge_index
    #x = self.layers[0](x.flatten())
    x = self.layer(x,e[[1,0],:])
    #x = self.layers[-1](x)
    return x
    #return nn.LogSoftmax(dim=1)(x)
    
  @staticmethod
  def _weight_reset(module):
    if isinstance(module, GAT) or isinstance(module, nn.Linear):
      module.reset_parameters()

  def reset_parameters(self):
    return self.layers.apply(type(self)._weight_reset)

class GAT_MAX(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, num_layers=0, graph_size=graph_size, act_fn=nn.ReLU()):
    super().__init__()
    self.act_fn = act_fn
    self.num_layers = num_layers
    self.layers = nn.ModuleList([])
    #self.layers.append(nn.Embedding(2, hidden_dim))
    self.layer=GAT(
        input_dim, hidden_dim, num_layers, output_dim, v2=True, jk='max'
    )
    #self.layers.append(nn.Linear(hidden_dim, output_dim))

  def forward(self, input_geom):
    x = input_geom.x
    e = input_geom.edge_index
    #x = self.layers[0](x.flatten())
    x = self.layer(x,e[[1,0],:])
    #x = self.layers[-1](x)
    return x
    #return nn.LogSoftmax(dim=1)(x)
    
  @staticmethod
  def _weight_reset(module):
    if isinstance(module, GAT) or isinstance(module, nn.Linear):
      module.reset_parameters()

  def reset_parameters(self):
    return self.layers.apply(type(self)._weight_reset)

In [ ]:
from torch_geometric.nn import GCN

class GCN_N(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, num_layers=0, graph_size=graph_size, act_fn=nn.ReLU()):
    super().__init__()
    self.act_fn = act_fn
    self.num_layers = num_layers
    self.layers = nn.ModuleList([])
    #self.layers.append(nn.Embedding(2, hidden_dim))
    self.layer = GCN(
        input_dim, hidden_dim, num_layers, output_dim, v2=True, jk=None
    )
    #self.layers.append(nn.Linear(hidden_dim, output_dim))

  def forward(self, input_geom):
    x = input_geom.x
    e = input_geom.edge_index
    #x = self.layers[0](x.flatten())
    x = self.layer(x,e[[1,0],:])
    #x = self.layers[-1](x)
    return x
    #return nn.LogSoftmax(dim=1)(x)
    
  @staticmethod
  def _weight_reset(module):
    if isinstance(module, GCN) or isinstance(module, nn.Linear):
      module.reset_parameters()

  def reset_parameters(self):
    return self.layers.apply(type(self)._weight_reset)

class GCN_CAT(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, num_layers=0, graph_size=graph_size, act_fn=nn.ReLU()):
    super().__init__()
    self.act_fn = act_fn
    self.num_layers = num_layers
    self.layers = nn.ModuleList([])
    #self.layers.append(nn.Embedding(2, hidden_dim))
    self.layer = GCN(
        input_dim, hidden_dim, num_layers, output_dim, v2=True, jk='cat'
    )
    #self.layers.append(nn.Linear(hidden_dim, output_dim))

  def forward(self, input_geom):
    x = input_geom.x
    e = input_geom.edge_index
    #x = self.layers[0](x.flatten())
    x = self.layer(x,e[[1,0],:])
    #x = self.layers[-1](x)
    return x
    #return nn.LogSoftmax(dim=1)(x)
    
  @staticmethod
  def _weight_reset(module):
    if isinstance(module, GCN) or isinstance(module, nn.Linear):
      module.reset_parameters()

  def reset_parameters(self):
    return self.layers.apply(type(self)._weight_reset)

class GCN_MAX(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, num_layers=0, graph_size=graph_size, act_fn=nn.ReLU()):
    super().__init__()
    self.act_fn = act_fn
    self.num_layers = num_layers
    self.layers = nn.ModuleList([])
    #self.layers.append(nn.Embedding(2, hidden_dim))
    self.layer=GCN(
        input_dim, hidden_dim, num_layers, output_dim, v2=True, jk='max'
    )
    #self.layers.append(nn.Linear(hidden_dim, output_dim))

  def forward(self, input_geom):
    x = input_geom.x
    e = input_geom.edge_index
    #x = self.layers[0](x.flatten())
    x = self.layer(x,e[[1,0],:])
    #x = self.layers[-1](x)
    return x
    #return nn.LogSoftmax(dim=1)(x)
    
  @staticmethod
  def _weight_reset(module):
    if isinstance(module, GCN) or isinstance(module, nn.Linear):
      module.reset_parameters()

  def reset_parameters(self):
    return self.layers.apply(type(self)._weight_reset)

In [ ]:
from torch_geometric.nn.norm import graph_size_norm
from operator import length_hint
def test(dataloader, model):
    """Test a model on some data."""
    global graph_size
    # Put the model in evaluation mode
    model.eval()
    
    # Get the number of datapoints
    size = len(dataloader.dataset)

    # The number of correct predictions
    correct = 0

    # We don't want to be computing the gradients
    with torch.no_grad():

        # Loop through the minibatches
        for data in dataloader:

            # Compute the model predictions
            result = model(data)
            length,dim = result.shape
            batch_size = int(length/graph_size)
            pred = torch.zeros((batch_size,dim))
            
            for i in range(length):
              if i % graph_size == 0:
                pred[0] = result[0]
            pred = result[torch.arange(batch_size)*graph_size]
            '''
            # Update with the number of correct predictions
            for i in range(batch_size):
              if data.y[i]+0.5> pred[i] and data.y[i]-0.5 <= pred[i]:
                correct += 1
            '''
            correct += (pred.argmax(dim=1)==data.y).sum()
            '''
            for i in range(batch_size):
              if pred[i] == data.y[i]:
                correct+=1
            '''
    # Compute the accuracy for the whole dataset and return it
    return correct / len(dataloader.dataset)

In [ ]:
def eval(dataset, model, loss_fn, optimiser, max_patience=5, num_splits=5,
                   batch_size=32, epochs=200, output_every=100, early_stop = 10):

  # Get the number of graphs
  size = len(dataset)

  # Construct a permuter which keeps paired graphs together
  graph_permuter = rng.permutation(np.arange(size))
  #graph_permuter = np.arange(size)
  
  # Use the permuter to shuffle the dataset
  shuffled_dataset = []
  for i in graph_permuter:
      shuffled_dataset.append(dataset[i])

  train_dataset = shuffled_dataset[:int(0.8*size)]
  test_dataset = shuffled_dataset[int(0.8*size):int(0.9*size)]
  valid_dataset = shuffled_dataset[int(0.9*size):]

  
  # Reset the parameter of the model before training
  model.reset_parameters()

  model.train()
  losses = []
  train_accuracies = []
  test_accuracies = []
  val_accuracies = []
  streaks = 0
  train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
  test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
  valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)
  for epoch in range(epochs):
    temp_loss = []
    for data in train_dataloader:
      optimizer.zero_grad()
      result = model(data)
      length, dim = result.shape
      batch_size = int(length/graph_size)
      # Compute the loss for this prediction
      pred = torch.zeros((batch_size,dim))
      #print(pred[0:10])
      if batch_size == 1:
        pred[0] = result[0]
      else:
        for i in range(length):
          if i % graph_size == 0:
            pred[int(i/graph_size)] = result[i]
      
      pred = result[torch.arange(batch_size)*graph_size]
      #pred.requires_grad_()
      loss = loss_fn(pred, data.y)
      #result.requires_grad_()
      #print(loss.requires_grad)
      #print(result.requires_grad)
      #print(pred.requires_grad)
      loss.requires_grad_()
      loss.backward()
      #print(loss.is_leaf)
      
      #print(loss.grad)
      #print(loss)
      #print(result.grad)
      optimizer.step()

      temp_loss.append(loss.item())
    losses.append(sum(temp_loss)/len(temp_loss))
    train_acc = test(train_dataloader, model)
    test_acc = test(test_dataloader, model)
    val_acc = test(valid_dataloader, model)
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    val_accuracies.append(val_acc)
    pickle1 = open("X.pickle","wb")
    pickle.dump([train_accuracies,test_accuracies,val_accuracies],pickle1)
    pickle1.close()
    if epoch % output_every ==0:
      print("===============Epoch"+str(epoch)+"==============")
      print(pred[0:5])
      print(pred.argmax(dim=1).float()[0:5])
      print(data.y[0:5])
      print("train acc: "+str(train_accuracies[-1]))
      print("test acc: "+str(test_accuracies[-1]))
      print("val acc: "+str(val_accuracies[-1]))
      print("train loss: "+str(losses[-1]))
      print("===================================")
      '''
      if epoch!=0:
        if (val_accuracies[-1]>val_accuracies[-11]):
          streaks += 1
        else:
          streaks = 0
      '''
    if streaks >= max_patience and train_accuracies[-1]>0.7:
      print("===============Epoch"+str(epoch)+"==============")
      print("train acc: "+str(train_accuracies[-1]))
      print("test acc: "+str(test_accuracies[-1]))
      print("val acc: "+str(val_accuracies[-1]))
      print("train loss: "+str(losses[-1]))
      print("===================================")
      files.download("X.pickle")
      break
  print("train acc: "+str(max(train_accuracies)))
  print("test acc: "+str(max(test_accuracies)))
  print("val acc: "+str(max(val_accuracies)))


In [ ]:
model = GCN_N(input_dim=1, hidden_dim=9, output_dim=9, num_layers=3).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lr=0.01, params=model.parameters())
eval(dataset, model1, loss_fn, optimizer, epochs=400, output_every=10, batch_size=1024)

===============Epoch0==============
tensor([[ 0.7464,  0.7241,  0.7359, -0.1351, -0.8409, -0.8902, -1.1736, -1.0024,
         -0.3276],
        [ 0.7706,  0.7597,  0.7724, -0.1320, -0.8825, -0.9257, -1.2252, -1.0485,
         -0.3258],
        [ 0.7464,  0.7241,  0.7359, -0.1351, -0.8409, -0.8902, -1.1736, -1.0024,
         -0.3276],
        [ 0.7463,  0.7247,  0.7364, -0.1353, -0.8416, -0.8903, -1.1737, -1.0029,
         -0.3272],
        [ 0.8333,  0.8348,  0.8512, -0.1203, -0.9703, -1.0109, -1.3481, -1.1515,
         -0.3319]], grad_fn=<SliceBackward0>)
tensor([0., 2., 0., 0., 2.])
tensor([0, 0, 2, 4, 0])
train acc: tensor(0.4668)
test acc: tensor(0.4681)
val acc: tensor(0.4611)
train loss: 1.9383614796858568
===============Epoch10==============
tensor([[ 2.2865,  1.4835,  1.4877,  0.6114,  0.0947, -1.2333, -2.1491, -3.7824,
         -3.5483],
        [ 2.4174,  1.5849,  1.5558,  0.6579,  0.0799, -1.3012, -2.2796, -4.0200,
         -3.7374],
        [ 2.2854,  1.4822,  1.4868,  0.61

KeyboardInterrupt: ignored

In [ ]:
model = GCN_MAX(input_dim=1, hidden_dim=9, output_dim=9, num_layers=3).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lr=0.01, params=model.parameters())
eval(dataset, model1, loss_fn, optimizer, epochs=400, output_every=10, batch_size=1024)

In [ ]:
model = GCN_CAT(input_dim=1, hidden_dim=9, output_dim=9, num_layers=3).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lr=0.01, params=model.parameters())
eval(dataset, model1, loss_fn, optimizer, epochs=400, output_every=10, batch_size=1024)

In [ ]:
model = GAT_N(input_dim=1, hidden_dim=9, output_dim=9, num_layers=3).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lr=0.01, params=model.parameters())
eval(dataset, model1, loss_fn, optimizer, epochs=400, output_every=10, batch_size=1024)

In [ ]:
model = GAT_MAX(input_dim=1, hidden_dim=9, output_dim=9, num_layers=3).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lr=0.01, params=model.parameters())
eval(dataset, model1, loss_fn, optimizer, epochs=400, output_every=10, batch_size=1024)

In [ ]:
model = GAT_CAT(input_dim=1, hidden_dim=9, output_dim=9, num_layers=3).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lr=0.01, params=model.parameters())
eval(dataset, model1, loss_fn, optimizer, epochs=400, output_every=10, batch_size=1024)